# Day 3 - ReAct Pattern with LangGraph
# ReAct 패턴을 활용한 추론 루프 구현

이 실습에서는 ReAct (Reasoning and Acting) 패턴을 LangGraph로 구현하고
Day 1 파인튜닝 모델을 활용한 도구 사용 에이전트를 구축합니다.

## ReAct 패턴이란?
- **Re**asoning (추론): 문제를 분석하고 계획을 세움
- **Act** (행동): 도구를 사용하여 정보를 수집하거나 작업을 수행
- **Observe** (관찰): 행동의 결과를 관찰하고 다음 단계를 결정

## 학습 목표
- ReAct 패턴의 핵심 개념 이해
- 도구(Tools) 시스템 구현
- 추론-행동-관찰 루프 구성
- Day 1 모델과 도구의 통합

In [ ]:
!pip install langchain langgraph transformers torch peft requests beautifulsoup4

In [ ]:
import os
import json
import time
import re
from typing import Dict, Any, TypedDict, List, Optional, Callable
from datetime import datetime
from dataclasses import dataclass
from enum import Enum

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolExecutor

# 추가 라이브러리 (도구 구현용)
import requests
from bs4 import BeautifulSoup
import math

print("라이브러리 import 완료")

## 1. Day 1 파인튜닝 모델 로드

In [ ]:
class Day1FinetunedLLM:
    def __init__(self, model_name="ryanu/my-exaone-raft-model"):
        self.model_name = model_name
        self.base_model = "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct"
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        print(f"🚀 Day 1 ReAct 모델 로딩: {model_name}")
        print(f"💾 디바이스: {self.device}")
        
        try:
            self.tokenizer = AutoTokenizer.from_pretrained(
                self.base_model,
                trust_remote_code=True
            )
            
            self.model = AutoModelForCausalLM.from_pretrained(
                self.base_model,
                torch_dtype=torch.float16,
                device_map="auto",
                trust_remote_code=True
            )
            
            try:
                self.model = PeftModel.from_pretrained(self.model, model_name)
                print("✅ 파인튜닝 어댑터 로드 성공")
            except Exception as e:
                print(f"⚠️ 파인튜닝 어댑터 로드 실패, 베이스 모델 사용: {e}")
                
        except Exception as e:
            print(f"⚠️ 모델 로드 실패, 데모 모드 활성화: {e}")
            self.model = None
            self.tokenizer = None
    
    def generate(self, prompt: str, max_length: int = 512, temperature: float = 0.7) -> str:
        """텍스트 생성"""
        if self.model is None:
            return f"[데모] {prompt[-100:]}...에 대한 ReAct 응답"
        
        try:
            inputs = self.tokenizer(
                prompt, 
                return_tensors="pt", 
                truncation=True, 
                max_length=2048
            ).to(self.device)
            
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_length=inputs['input_ids'].shape[1] + max_length,
                    temperature=temperature,
                    do_sample=True,
                    pad_token_id=self.tokenizer.eos_token_id
                )
            
            generated_text = self.tokenizer.decode(
                outputs[0][inputs['input_ids'].shape[1]:], 
                skip_special_tokens=True
            )
            
            return generated_text.strip()
            
        except Exception as e:
            return f"생성 오류: {str(e)}"

# 글로벌 LLM 인스턴스
llm = Day1FinetunedLLM()
print("🎯 ReAct LLM 초기화 완료")

## 2. 도구(Tools) 시스템 구현

In [ ]:
@dataclass
class Tool:
    """도구 클래스"""
    name: str
    description: str
    function: Callable
    parameters: Dict[str, Any]

class ToolRegistry:
    """도구 레지스트리"""
    
    def __init__(self):
        self.tools = {}
        self._register_default_tools()
    
    def register_tool(self, tool: Tool):
        """도구 등록"""
        self.tools[tool.name] = tool
        print(f"🔧 도구 등록: {tool.name}")
    
    def get_tool(self, name: str) -> Optional[Tool]:
        """도구 가져오기"""
        return self.tools.get(name)
    
    def list_tools(self) -> List[str]:
        """도구 목록"""
        return list(self.tools.keys())
    
    def get_tools_description(self) -> str:
        """도구 설명 텍스트"""
        descriptions = []
        for name, tool in self.tools.items():
            descriptions.append(f"- {name}: {tool.description}")
        return "\n".join(descriptions)
    
    def execute_tool(self, tool_name: str, **kwargs) -> Dict[str, Any]:
        """도구 실행"""
        tool = self.get_tool(tool_name)
        if not tool:
            return {
                "success": False,
                "error": f"도구 '{tool_name}'을 찾을 수 없습니다",
                "result": None
            }
        
        try:
            result = tool.function(**kwargs)
            return {
                "success": True,
                "error": None,
                "result": result
            }
        except Exception as e:
            return {
                "success": False,
                "error": str(e),
                "result": None
            }
    
    def _register_default_tools(self):
        """기본 도구들 등록"""
        
        # 계산기 도구
        def calculator(expression: str) -> str:
            """수학 계산 수행"""
            try:
                # 안전한 계산을 위한 검증
                allowed_chars = set('0123456789+-*/.() ')
                if not all(c in allowed_chars for c in expression):
                    return f"오류: 허용되지 않은 문자가 포함되어 있습니다: {expression}"
                
                # 기본 수학 함수들 추가
                safe_dict = {
                    "__builtins__": {},
                    "abs": abs,
                    "round": round,
                    "min": min,
                    "max": max,
                    "pow": pow,
                    "sqrt": math.sqrt,
                    "sin": math.sin,
                    "cos": math.cos,
                    "tan": math.tan,
                    "pi": math.pi,
                    "e": math.e
                }
                
                result = eval(expression, safe_dict)
                return f"계산 결과: {expression} = {result}"
                
            except Exception as e:
                return f"계산 오류: {str(e)}"
        
        self.register_tool(Tool(
            name="calculator",
            description="수학 계산을 수행합니다. 예: calculator(expression='2+2*3')",
            function=calculator,
            parameters={"expression": "수학 표현식 문자열"}
        ))
        
        # 검색 도구 (시뮬레이션)
        def web_search(query: str, num_results: int = 3) -> str:
            """웹 검색 시뮬레이션"""
            # 실제 환경에서는 실제 검색 API를 사용
            search_results = [
                f"'{query}'에 대한 검색 결과 {i+1}: 관련 정보가 포함된 웹사이트입니다."
                for i in range(num_results)
            ]
            
            return f"검색어 '{query}' 결과:\n" + "\n".join(search_results)
        
        self.register_tool(Tool(
            name="web_search",
            description="웹에서 정보를 검색합니다. 예: web_search(query='인공지능', num_results=3)",
            function=web_search,
            parameters={"query": "검색할 키워드", "num_results": "반환할 결과 수 (기본값: 3)"}
        ))
        
        # 현재 시간 도구
        def get_current_time() -> str:
            """현재 시간 반환"""
            now = datetime.now()
            return f"현재 시간: {now.strftime('%Y년 %m월 %d일 %H시 %M분 %S초')}"
        
        self.register_tool(Tool(
            name="get_time",
            description="현재 시간을 알려줍니다. 예: get_time()",
            function=get_current_time,
            parameters={}
        ))
        
        # 텍스트 분석 도구
        def text_analyzer(text: str) -> str:
            """텍스트 분석"""
            word_count = len(text.split())
            char_count = len(text)
            sentence_count = len([s for s in text.split('.') if s.strip()])
            
            # 간단한 감정 분석
            positive_words = ['좋은', '훌륭한', '멋진', '최고', '행복', '기쁜', '사랑']
            negative_words = ['나쁜', '싫은', '화난', '슬픈', '실망', '짜증', '걱정']
            
            positive_count = sum(1 for word in positive_words if word in text)
            negative_count = sum(1 for word in negative_words if word in text)
            
            if positive_count > negative_count:
                sentiment = "긍정적"
            elif negative_count > positive_count:
                sentiment = "부정적"
            else:
                sentiment = "중립적"
            
            return f"""텍스트 분석 결과:
- 글자 수: {char_count}
- 단어 수: {word_count}
- 문장 수: {sentence_count}
- 감정: {sentiment}
- 긍정 단어: {positive_count}개
- 부정 단어: {negative_count}개"""
        
        self.register_tool(Tool(
            name="text_analyzer",
            description="텍스트를 분석하여 통계와 감정을 알려줍니다. 예: text_analyzer(text='분석할 텍스트')",
            function=text_analyzer,
            parameters={"text": "분석할 텍스트"}
        ))
        
        # 번역 도구 (시뮬레이션)
        def simple_translator(text: str, target_language: str = "영어") -> str:
            """간단한 번역 시뮬레이션"""
            translations = {
                "영어": {
                    "안녕하세요": "Hello",
                    "감사합니다": "Thank you",
                    "인공지능": "Artificial Intelligence",
                    "머신러닝": "Machine Learning",
                    "딥러닝": "Deep Learning"
                },
                "일본어": {
                    "안녕하세요": "こんにちは",
                    "감사합니다": "ありがとうございます",
                    "인공지능": "人工知能",
                    "머신러닝": "機械学習",
                    "딥러닝": "深層学習"
                }
            }
            
            if target_language in translations:
                for korean, translated in translations[target_language].items():
                    if korean in text:
                        text = text.replace(korean, translated)
            
            return f"번역 결과 ({target_language}): {text}"
        
        self.register_tool(Tool(
            name="translator",
            description="텍스트를 다른 언어로 번역합니다. 예: translator(text='안녕하세요', target_language='영어')",
            function=simple_translator,
            parameters={"text": "번역할 텍스트", "target_language": "목표 언어 (영어, 일본어)"}
        ))

# 도구 레지스트리 초기화
tool_registry = ToolRegistry()
print(f"✅ 도구 레지스트리 초기화 완료 ({len(tool_registry.list_tools())}개 도구)")
print(f"등록된 도구: {', '.join(tool_registry.list_tools())}")

## 3. ReAct 상태 정의

In [ ]:
class ReActStep(Enum):
    THINKING = "thinking"
    ACTION = "action"
    OBSERVATION = "observation"
    FINAL_ANSWER = "final_answer"

class ReActState(TypedDict):
    # 기본 입력/출력
    user_question: str
    final_answer: str
    
    # ReAct 루프 상태
    current_step: str  # ReActStep의 value
    iteration: int
    max_iterations: int
    
    # 추론 과정
    thought: str
    action: str
    action_input: str
    observation: str
    
    # 히스토리
    thoughts_history: List[str]
    actions_history: List[Dict[str, Any]]
    observations_history: List[str]
    
    # 메타데이터
    available_tools: List[str]
    is_complete: bool
    confidence: float
    reasoning_trace: str
    
    # 에러 처리
    errors: List[str]

print("✅ ReAct 상태 정의 완료")
print(f"ReAct 단계: {[step.value for step in ReActStep]}")

## 4. ReAct 노드 구현

In [ ]:
class ReActAgent:
    """ReAct 에이전트 클래스"""
    
    def __init__(self, tool_registry: ToolRegistry):
        self.tool_registry = tool_registry
        self.llm = llm
    
    def think(self, state: ReActState) -> ReActState:
        """추론 단계: 현재 상황을 분석하고 다음 행동을 계획"""
        print(f"🤔 추론 단계 (반복 {state['iteration']}회차)")
        
        user_question = state["user_question"]
        thoughts_history = state["thoughts_history"]
        observations_history = state["observations_history"]
        available_tools = state["available_tools"]
        
        # 추론 프롬프트 구성
        tools_description = self.tool_registry.get_tools_description()
        
        history_context = ""
        if thoughts_history:
            history_context = "\n이전 추론과 관찰:\n"
            for i, (thought, obs) in enumerate(zip(thoughts_history, observations_history)):
                history_context += f"추론 {i+1}: {thought}\n"
                if obs:
                    history_context += f"관찰 {i+1}: {obs}\n"
        
        prompt = f"""당신은 ReAct 패턴을 따르는 AI 에이전트입니다. 주어진 질문에 답하기 위해 단계적으로 추론하고 필요한 도구를 사용해야 합니다.

사용 가능한 도구들:
{tools_description}

질문: {user_question}
{history_context}

현재 상황을 분석하고 다음에 무엇을 해야 할지 추론해주세요.

추론 형식:
1. 현재 상황 분석
2. 목표 달성을 위해 필요한 정보나 작업
3. 다음 행동 계획 (도구 사용 또는 최종 답변)

추론:"""
        
        thought = self.llm.generate(prompt, max_length=300, temperature=0.7)
        
        print(f"💭 추론 결과: {thought[:100]}...")
        
        updated_thoughts = thoughts_history + [thought]
        
        return {
            **state,
            "current_step": ReActStep.THINKING.value,
            "thought": thought,
            "thoughts_history": updated_thoughts
        }
    
    def act(self, state: ReActState) -> ReActState:
        """행동 단계: 도구를 선택하고 실행"""
        print("⚡ 행동 단계")
        
        thought = state["thought"]
        user_question = state["user_question"]
        available_tools = state["available_tools"]
        
        # 행동 결정 프롬프트
        tools_list = ", ".join(available_tools)
        
        action_prompt = f"""다음 추론을 바탕으로 구체적인 행동을 결정하세요.

추론: {thought}
질문: {user_question}
사용 가능한 도구: {tools_list}

다음 중 하나를 선택하세요:
1. 도구 사용: "USE_TOOL: [도구명] [입력값]"
2. 최종 답변: "FINAL_ANSWER: [답변]"

예시:
- USE_TOOL: calculator 2+2*3
- USE_TOOL: web_search 인공지능
- FINAL_ANSWER: 계산 결과는 8입니다.

행동:"""
        
        action_decision = self.llm.generate(action_prompt, max_length=100, temperature=0.3)
        
        # 행동 파싱
        action_decision = action_decision.strip()
        
        if action_decision.startswith("USE_TOOL:"):
            # 도구 사용
            tool_command = action_decision[9:].strip()  # "USE_TOOL: " 제거
            parts = tool_command.split(" ", 1)
            
            if len(parts) >= 1:
                tool_name = parts[0]
                tool_input = parts[1] if len(parts) > 1 else ""
                
                action = f"도구 사용: {tool_name}"
                action_input = tool_input
            else:
                action = "도구 사용 실패"
                action_input = "도구명을 파싱할 수 없음"
                
        elif action_decision.startswith("FINAL_ANSWER:"):
            # 최종 답변
            final_answer = action_decision[13:].strip()  # "FINAL_ANSWER: " 제거
            action = "최종 답변"
            action_input = final_answer
            
        else:
            # 기본값
            action = "불명확한 행동"
            action_input = action_decision
        
        print(f"🎯 선택된 행동: {action}")
        if action_input:
            print(f"📥 행동 입력: {action_input[:50]}...")
        
        # 행동 기록 업데이트
        action_record = {
            "iteration": state["iteration"],
            "action": action,
            "input": action_input,
            "timestamp": datetime.now().isoformat()
        }
        updated_actions = state["actions_history"] + [action_record]
        
        return {
            **state,
            "current_step": ReActStep.ACTION.value,
            "action": action,
            "action_input": action_input,
            "actions_history": updated_actions
        }
    
    def observe(self, state: ReActState) -> ReActState:
        """관찰 단계: 행동의 결과를 관찰하고 기록"""
        print("👁 관찰 단계")
        
        action = state["action"]
        action_input = state["action_input"]
        
        observation = ""
        is_complete = False
        
        if action == "최종 답변":
            observation = f"최종 답변 제공됨: {action_input}"
            is_complete = True
            
        elif action.startswith("도구 사용:"):
            # 도구 실행
            tool_name = action.split(": ")[1]
            
            if tool_name in self.tool_registry.list_tools():
                # 도구 파라미터 파싱 (간단한 방식)
                tool_params = self._parse_tool_params(tool_name, action_input)
                
                # 도구 실행
                result = self.tool_registry.execute_tool(tool_name, **tool_params)
                
                if result["success"]:
                    observation = f"도구 '{tool_name}' 실행 성공:\n{result['result']}"
                else:
                    observation = f"도구 '{tool_name}' 실행 실패: {result['error']}"
            else:
                observation = f"도구 '{tool_name}'을 찾을 수 없습니다."
        
        else:
            observation = f"알 수 없는 행동: {action}"
        
        print(f"📊 관찰 결과: {observation[:100]}...")
        
        # 관찰 기록 업데이트
        updated_observations = state["observations_history"] + [observation]
        
        # 추론 추적 업데이트
        reasoning_step = f"""반복 {state['iteration']}:
추론: {state['thought']}
행동: {action}
관찰: {observation}
---"""
        
        updated_trace = state["reasoning_trace"] + "\n" + reasoning_step
        
        return {
            **state,
            "current_step": ReActStep.OBSERVATION.value,
            "observation": observation,
            "observations_history": updated_observations,
            "is_complete": is_complete,
            "reasoning_trace": updated_trace,
            "iteration": state["iteration"] + 1
        }
    
    def _parse_tool_params(self, tool_name: str, action_input: str) -> Dict[str, Any]:
        """도구 파라미터 파싱"""
        # 간단한 파라미터 파싱 (실제로는 더 정교한 파싱 필요)
        if tool_name == "calculator":
            return {"expression": action_input}
        elif tool_name == "web_search":
            return {"query": action_input, "num_results": 3}
        elif tool_name == "text_analyzer":
            return {"text": action_input}
        elif tool_name == "translator":
            parts = action_input.split(" ", 1)
            if len(parts) > 1:
                return {"text": parts[0], "target_language": parts[1]}
            else:
                return {"text": action_input, "target_language": "영어"}
        elif tool_name == "get_time":
            return {}
        else:
            return {"input": action_input}
    
    def finalize_answer(self, state: ReActState) -> ReActState:
        """최종 답변 생성"""
        print("🎯 최종 답변 생성")
        
        if state["action"] == "최종 답변":
            final_answer = state["action_input"]
        else:
            # 추론 과정을 바탕으로 최종 답변 생성
            user_question = state["user_question"]
            reasoning_trace = state["reasoning_trace"]
            
            final_prompt = f"""다음 추론 과정을 바탕으로 사용자의 질문에 대한 최종 답변을 작성하세요.

질문: {user_question}

추론 과정:
{reasoning_trace}

위의 추론과 도구 사용 결과를 종합하여 명확하고 도움이 되는 최종 답변을 제공하세요.

최종 답변:"""
            
            final_answer = self.llm.generate(final_prompt, max_length=400, temperature=0.5)
        
        # 신뢰도 계산
        confidence = self._calculate_confidence(state)
        
        print(f"✅ 최종 답변 완성 (신뢰도: {confidence:.1%})")
        
        return {
            **state,
            "current_step": ReActStep.FINAL_ANSWER.value,
            "final_answer": final_answer,
            "confidence": confidence,
            "is_complete": True
        }
    
    def _calculate_confidence(self, state: ReActState) -> float:
        """신뢰도 계산"""
        base_confidence = 0.5
        
        # 성공적인 도구 사용 횟수
        successful_actions = 0
        for obs in state["observations_history"]:
            if "실행 성공" in obs:
                successful_actions += 1
        
        # 도구 사용 성공률에 따른 신뢰도 조정
        if successful_actions > 0:
            base_confidence += min(successful_actions * 0.2, 0.4)
        
        # 반복 횟수에 따른 조정
        if state["iteration"] <= 3:
            base_confidence += 0.1
        
        return min(base_confidence, 0.95)

# ReAct 에이전트 초기화
react_agent = ReActAgent(tool_registry)
print("✅ ReAct 에이전트 초기화 완료")

## 5. ReAct 워크플로 구성

In [ ]:
def should_continue(state: ReActState) -> str:
    """루프 계속 여부 결정"""
    if state["is_complete"]:
        return "finalize"
    elif state["iteration"] >= state["max_iterations"]:
        return "finalize"
    else:
        return "think"

# ReAct 워크플로 구성
react_workflow = StateGraph(ReActState)

# 노드 추가
react_workflow.add_node("think", react_agent.think)
react_workflow.add_node("act", react_agent.act)
react_workflow.add_node("observe", react_agent.observe)
react_workflow.add_node("finalize", react_agent.finalize_answer)

# 엣지 연결
react_workflow.set_entry_point("think")
react_workflow.add_edge("think", "act")
react_workflow.add_edge("act", "observe")

# 조건부 엣지 (루프 또는 종료)
react_workflow.add_conditional_edges(
    "observe",
    should_continue,
    {
        "think": "think",      # 루프 계속
        "finalize": "finalize" # 최종화
    }
)

react_workflow.add_edge("finalize", END)

# ReAct 워크플로 컴파일
react_app = react_workflow.compile()

print("✅ ReAct 워크플로 구성 완료")
print("흐름: think → act → observe → [think (루프) / finalize] → END")

## 6. ReAct 시스템 테스트

In [ ]:
def test_react_agent(question: str, max_iterations: int = 5):
    """ReAct 에이전트 테스트"""
    print(f"\n🚀 ReAct 에이전트 테스트")
    print(f"❓ 질문: {question}")
    print(f"🔄 최대 반복: {max_iterations}회")
    print("=" * 80)
    
    # 초기 상태 설정
    initial_state = {
        "user_question": question,
        "final_answer": "",
        "current_step": ReActStep.THINKING.value,
        "iteration": 1,
        "max_iterations": max_iterations,
        "thought": "",
        "action": "",
        "action_input": "",
        "observation": "",
        "thoughts_history": [],
        "actions_history": [],
        "observations_history": [],
        "available_tools": tool_registry.list_tools(),
        "is_complete": False,
        "confidence": 0.0,
        "reasoning_trace": "",
        "errors": []
    }
    
    try:
        final_state = react_app.invoke(initial_state)
        
        print("\n📤 ReAct 결과")
        print("=" * 80)
        print(f"🎯 최종 답변: {final_state['final_answer']}")
        print(f"🔄 총 반복 횟수: {final_state['iteration'] - 1}회")
        print(f"📊 신뢰도: {final_state['confidence']:.1%}")
        print(f"🛠 사용된 도구: {len([a for a in final_state['actions_history'] if '도구 사용' in a['action']])}개")
        
        return final_state
        
    except Exception as e:
        print(f"❌ ReAct 실행 중 오류: {e}")
        return None

# 테스트 케이스들
react_test_cases = [
    "25 곱하기 4는 얼마인가요?",
    "현재 시간을 알려주고, 그 정보를 분석해주세요",
    "'안녕하세요 좋은 하루입니다'라는 텍스트를 영어로 번역해주세요",
    "인공지능에 대해 검색하고 그 결과를 요약해주세요",
    "100을 3으로 나눈 결과를 분석해주세요"
]

# 첫 번째 테스트 실행
result = test_react_agent(react_test_cases[0], max_iterations=3)

## 7. 상세한 추론 과정 시각화

In [ ]:
def visualize_react_process(final_state: ReActState):
    """ReAct 추론 과정 상세 시각화"""
    if not final_state:
        print("❌ 시각화할 결과가 없습니다.")
        return
    
    print("\n🔍 ReAct 추론 과정 상세 분석")
    print("=" * 100)
    
    question = final_state["user_question"]
    iterations = final_state["iteration"] - 1
    
    print(f"📋 질문: {question}")
    print(f"🔄 총 반복: {iterations}회")
    print(f"🎯 최종 신뢰도: {final_state['confidence']:.1%}")
    
    # 단계별 상세 분석
    thoughts = final_state["thoughts_history"]
    actions = final_state["actions_history"]
    observations = final_state["observations_history"]
    
    print("\n📊 단계별 ReAct 과정:")
    
    for i in range(len(thoughts)):
        print(f"\n🔄 반복 {i+1}:")
        print("-" * 60)
        
        # Thought
        print(f"💭 추론 (Think):")
        thought = thoughts[i] if i < len(thoughts) else "없음"
        print(f"   {thought[:200]}{'...' if len(thought) > 200 else ''}")
        
        # Action
        if i < len(actions):
            action_info = actions[i]
            print(f"⚡ 행동 (Act):")
            print(f"   행동: {action_info['action']}")
            if action_info['input']:
                print(f"   입력: {action_info['input'][:100]}{'...' if len(action_info['input']) > 100 else ''}")
        
        # Observation
        if i < len(observations):
            observation = observations[i]
            print(f"👁 관찰 (Observe):")
            print(f"   {observation[:200]}{'...' if len(observation) > 200 else ''}")
    
    # 도구 사용 통계
    print("\n🛠 도구 사용 통계:")
    tool_usage = {}
    for action in actions:
        if "도구 사용" in action["action"]:
            tool_name = action["action"].split(": ")[1]
            tool_usage[tool_name] = tool_usage.get(tool_name, 0) + 1
    
    if tool_usage:
        for tool, count in tool_usage.items():
            print(f"   {tool}: {count}회 사용")
    else:
        print("   사용된 도구 없음")
    
    # 성능 분석
    print("\n📈 성능 분석:")
    successful_actions = sum(1 for obs in observations if "실행 성공" in obs)
    total_actions = len([a for a in actions if "도구 사용" in a["action"]])
    
    if total_actions > 0:
        success_rate = successful_actions / total_actions * 100
        print(f"   도구 실행 성공률: {success_rate:.1f}% ({successful_actions}/{total_actions})")
    
    print(f"   추론 효율성: {iterations}회 반복으로 완료")
    print(f"   최종 완성도: {'✅ 완료' if final_state['is_complete'] else '⚠️ 미완료'}")

# 결과 시각화
if result:
    visualize_react_process(result)

## 8. 다양한 시나리오 종합 테스트

In [ ]:
def comprehensive_react_test():
    """다양한 시나리오의 종합 테스트"""
    print("🧪 ReAct 에이전트 종합 테스트")
    print("=" * 100)
    
    test_results = []
    
    for i, question in enumerate(react_test_cases, 1):
        print(f"\n🎯 테스트 {i}/{len(react_test_cases)}")
        print(f"❓ 질문: {question}")
        print("-" * 80)
        
        start_time = time.time()
        result = test_react_agent(question, max_iterations=4)
        end_time = time.time()
        
        if result:
            test_results.append({
                "question": question,
                "success": result["is_complete"],
                "iterations": result["iteration"] - 1,
                "confidence": result["confidence"],
                "tools_used": len([a for a in result["actions_history"] if "도구 사용" in a["action"]]),
                "processing_time": end_time - start_time,
                "answer_length": len(result["final_answer"])
            })
            
            print(f"✅ 완료 - {result['iteration']-1}회 반복, 신뢰도: {result['confidence']:.1%}")
        else:
            test_results.append({
                "question": question,
                "success": False,
                "iterations": 0,
                "confidence": 0,
                "tools_used": 0,
                "processing_time": end_time - start_time,
                "answer_length": 0
            })
            print("❌ 실패")
    
    # 결과 분석
    print("\n\n📊 종합 테스트 결과 분석")
    print("=" * 100)
    
    total_tests = len(test_results)
    successful_tests = sum(1 for r in test_results if r["success"])
    success_rate = successful_tests / total_tests * 100
    
    avg_iterations = sum(r["iterations"] for r in test_results) / total_tests
    avg_confidence = sum(r["confidence"] for r in test_results if r["success"]) / max(successful_tests, 1)
    avg_tools_used = sum(r["tools_used"] for r in test_results) / total_tests
    avg_processing_time = sum(r["processing_time"] for r in test_results) / total_tests
    
    print(f"📈 성공률: {success_rate:.1f}% ({successful_tests}/{total_tests})")
    print(f"🔄 평균 반복 횟수: {avg_iterations:.1f}회")
    print(f"🎯 평균 신뢰도: {avg_confidence:.1%}")
    print(f"🛠 평균 도구 사용: {avg_tools_used:.1f}개")
    print(f"⏱️ 평균 처리 시간: {avg_processing_time:.2f}초")
    
    # 상세 결과
    print("\n📋 상세 결과:")
    for i, result in enumerate(test_results, 1):
        status = "✅" if result["success"] else "❌"
        print(f"[{i}] {status} {result['question'][:50]}...")
        print(f"    반복: {result['iterations']}회, 도구: {result['tools_used']}개, 신뢰도: {result['confidence']:.1%}")
    
    return test_results

# 종합 테스트 실행
comprehensive_results = comprehensive_react_test()

## 9. ReAct vs 기본 생성 비교

In [ ]:
def compare_react_vs_basic(question: str):
    """ReAct 방식과 기본 생성 방식 비교"""
    print(f"\n⚔️ ReAct vs 기본 생성 비교")
    print(f"❓ 질문: {question}")
    print("=" * 80)
    
    # 1. 기본 생성 방식
    print("\n📝 기본 생성 방식:")
    basic_prompt = f"다음 질문에 답변해주세요: {question}"
    basic_start = time.time()
    basic_answer = llm.generate(basic_prompt, max_length=300, temperature=0.7)
    basic_time = time.time() - basic_start
    
    print(f"답변: {basic_answer}")
    print(f"처리 시간: {basic_time:.2f}초")
    
    # 2. ReAct 방식
    print("\n🔄 ReAct 방식:")
    react_start = time.time()
    react_result = test_react_agent(question, max_iterations=3)
    react_time = time.time() - react_start
    
    if react_result:
        print(f"답변: {react_result['final_answer']}")
        print(f"처리 시간: {react_time:.2f}초")
        print(f"사용 도구: {len([a for a in react_result['actions_history'] if '도구 사용' in a['action']])}개")
        print(f"반복 횟수: {react_result['iteration']-1}회")
    else:
        print("ReAct 방식 실행 실패")
        return
    
    # 3. 비교 분석
    print("\n📊 비교 분석:")
    print(f"처리 시간: ReAct {react_time:.2f}초 vs 기본 {basic_time:.2f}초")
    print(f"답변 길이: ReAct {len(react_result['final_answer'])}자 vs 기본 {len(basic_answer)}자")
    
    # 정확성 평가 (간단한 휴리스틱)
    react_accuracy = "높음" if react_result['confidence'] > 0.7 else "보통"
    basic_accuracy = "보통"  # 기본값
    
    print(f"예상 정확성: ReAct {react_accuracy} vs 기본 {basic_accuracy}")
    
    # 장단점 분석
    print("\n💡 장단점 분석:")
    print("ReAct 방식:")
    print("  ✅ 도구 활용으로 정확한 정보 제공")
    print("  ✅ 단계별 추론으로 투명성 확보")
    print("  ⚠️ 처리 시간이 더 길 수 있음")
    print("  ⚠️ 복잡한 워크플로 관리 필요")
    
    print("기본 생성 방식:")
    print("  ✅ 빠른 응답 시간")
    print("  ✅ 간단한 구현")
    print("  ⚠️ 외부 정보 활용 제한")
    print("  ⚠️ 추론 과정 불투명")

# 비교 테스트
comparison_questions = [
    "100 곱하기 25는 얼마인가요?",
    "현재 시간은 몇 시인가요?",
]

for question in comparison_questions:
    compare_react_vs_basic(question)

## 10. 실습 요약 및 다음 단계

In [ ]:
print("""
🎓 ReAct 패턴 실습 완료!

📚 핵심 학습 내용:
✅ ReAct (Reasoning + Acting) 패턴 이해
✅ 도구(Tools) 시스템 구현
✅ Think → Act → Observe 루프 구성
✅ Day 1 파인튜닝 모델과 도구 통합
✅ 조건부 루프 및 종료 로직
✅ 추론 과정 추적 및 시각화
✅ 성능 분석 및 비교 평가

🛠 구현한 도구들:
• 🧮 calculator: 수학 계산
• 🔍 web_search: 웹 검색 (시뮬레이션)
• ⏰ get_time: 현재 시간
• 📊 text_analyzer: 텍스트 분석
• 🌐 translator: 간단한 번역

🔄 ReAct 패턴의 핵심 장점:
1. 📝 투명한 추론 과정
2. 🛠 외부 도구 활용 능력
3. 🔄 반복적 개선
4. 🎯 목표 지향적 행동
5. 📊 과정 추적 및 분석 가능

💡 실제 활용 시나리오:
• 🤖 AI 어시스턴트 (계산, 검색, 분석)
• 🔬 연구 도구 (데이터 수집 및 분석)
• 📚 교육용 튜터 (단계별 문제 해결)
• 🏢 업무 자동화 (다단계 작업 처리)
• 🛠 개발 도구 (코드 분석 및 생성)

🚀 다음 실습 내용:
• 03-advanced-rag.ipynb: 고급 RAG + ReAct 통합
• 04-text2sql.ipynb: SQL 생성 및 실행
• 05-mcp-integration.ipynb: 더 많은 외부 도구 통합
• 06-gradio-ui.ipynb: UI로 ReAct 시스템 래핑

🔬 고급 확장 아이디어:
- 더 정교한 도구 파라미터 파싱
- 병렬 도구 실행
- 도구 체이닝 (한 도구의 출력을 다른 도구의 입력으로)
- 동적 도구 발견 및 등록
- 에러 복구 및 재시도 메커니즘
""")

# 최종 통계
if 'comprehensive_results' in locals():
    successful_tests = sum(1 for r in comprehensive_results if r["success"])
    total_tests = len(comprehensive_results)
    
    print(f"\n📊 최종 실행 통계:")
    print(f"• 총 테스트: {total_tests}개")
    print(f"• 성공: {successful_tests}개")
    print(f"• 성공률: {successful_tests/total_tests*100:.1f}%")
    
    if comprehensive_results:
        avg_tools = sum(r["tools_used"] for r in comprehensive_results) / total_tests
        avg_iterations = sum(r["iterations"] for r in comprehensive_results) / total_tests
        print(f"• 평균 도구 사용: {avg_tools:.1f}개")
        print(f"• 평균 반복 횟수: {avg_iterations:.1f}회")

print("\n🎉 ReAct 패턴 마스터 완료! 다음 단계로 진행하세요.")